In [1]:

import os.path as osp
import time

import torch
import torch.nn.functional as F
from torch.nn import Parameter
from tqdm import tqdm

from torch_geometric.datasets import RelLinkPredDataset
from torch_geometric.nn import GAE, RGCNConv


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

path = 'rdata'
dataset = RelLinkPredDataset(path, 'FB15k-237')
data = dataset[0].to(device)


class RGCNEncoder(torch.nn.Module):
    def __init__(self, num_nodes, hidden_channels, num_relations):
        super().__init__()
        self.node_emb = Parameter(torch.empty(num_nodes, hidden_channels))
        self.conv1 = RGCNConv(hidden_channels, hidden_channels, num_relations,
                              num_blocks=5)
        self.conv2 = RGCNConv(hidden_channels, hidden_channels, num_relations,
                              num_blocks=5)
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.node_emb)
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, edge_index, edge_type):
        x = self.node_emb
        x = self.conv1(x, edge_index, edge_type).relu_()
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, edge_index, edge_type)
        return x


class DistMultDecoder(torch.nn.Module):
    def __init__(self, num_relations, hidden_channels):
        super().__init__()
        self.rel_emb = Parameter(torch.empty(num_relations, hidden_channels))
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.rel_emb)

    def forward(self, z, edge_index, edge_type):
        z_src, z_dst = z[edge_index[0]], z[edge_index[1]]
        rel = self.rel_emb[edge_type]
        return torch.sum(z_src * rel * z_dst, dim=1)


model = GAE(
    RGCNEncoder(data.num_nodes, 500, dataset.num_relations),
    DistMultDecoder(dataset.num_relations // 2, 500),
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


def negative_sampling(edge_index, num_nodes):
    # Sample edges by corrupting either the subject or the object of each edge.
    mask_1 = torch.rand(edge_index.size(1)) < 0.5
    mask_2 = ~mask_1

    neg_edge_index = edge_index.clone()
    neg_edge_index[0, mask_1] = torch.randint(num_nodes, (mask_1.sum(), ),
                                              device=neg_edge_index.device)
    neg_edge_index[1, mask_2] = torch.randint(num_nodes, (mask_2.sum(), ),
                                              device=neg_edge_index.device)
    return neg_edge_index


def train():
    model.train()
    optimizer.zero_grad()

    z = model.encode(data.edge_index, data.edge_type)

    pos_out = model.decode(z, data.train_edge_index, data.train_edge_type)

    neg_edge_index = negative_sampling(data.train_edge_index, data.num_nodes)
    neg_out = model.decode(z, neg_edge_index, data.train_edge_type)

    out = torch.cat([pos_out, neg_out])
    gt = torch.cat([torch.ones_like(pos_out), torch.zeros_like(neg_out)])
    cross_entropy_loss = F.binary_cross_entropy_with_logits(out, gt)
    reg_loss = z.pow(2).mean() + model.decoder.rel_emb.pow(2).mean()
    loss = cross_entropy_loss + 1e-2 * reg_loss

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.)
    optimizer.step()

    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    z = model.encode(data.edge_index, data.edge_type)

    valid_mrr = compute_mrr(z, data.valid_edge_index, data.valid_edge_type)
    test_mrr = compute_mrr(z, data.test_edge_index, data.test_edge_type)

    return valid_mrr, test_mrr


@torch.no_grad()
def compute_rank(ranks):
    # fair ranking prediction as the average
    # of optimistic and pessimistic ranking
    true = ranks[0]
    optimistic = (ranks > true).sum() + 1
    pessimistic = (ranks >= true).sum()
    return (optimistic + pessimistic).float() * 0.5


@torch.no_grad()
def compute_mrr(z, edge_index, edge_type):
    ranks = []
    for i in tqdm(range(edge_type.numel())):
        (src, dst), rel = edge_index[:, i], edge_type[i]

        # Try all nodes as tails, but delete true triplets:
        tail_mask = torch.ones(data.num_nodes, dtype=torch.bool)
        for (heads, tails), types in [
            (data.train_edge_index, data.train_edge_type),
            (data.valid_edge_index, data.valid_edge_type),
            (data.test_edge_index, data.test_edge_type),
        ]:
            tail_mask[tails[(heads == src) & (types == rel)]] = False

        tail = torch.arange(data.num_nodes)[tail_mask]
        tail = torch.cat([torch.tensor([dst]), tail])
        head = torch.full_like(tail, fill_value=src)
        eval_edge_index = torch.stack([head, tail], dim=0)
        eval_edge_type = torch.full_like(tail, fill_value=rel)

        out = model.decode(z, eval_edge_index, eval_edge_type)
        rank = compute_rank(out)
        ranks.append(rank)

        # Try all nodes as heads, but delete true triplets:
        head_mask = torch.ones(data.num_nodes, dtype=torch.bool)
        for (heads, tails), types in [
            (data.train_edge_index, data.train_edge_type),
            (data.valid_edge_index, data.valid_edge_type),
            (data.test_edge_index, data.test_edge_type),
        ]:
            head_mask[heads[(tails == dst) & (types == rel)]] = False

        head = torch.arange(data.num_nodes)[head_mask]
        head = torch.cat([torch.tensor([src]), head])
        tail = torch.full_like(head, fill_value=dst)
        eval_edge_index = torch.stack([head, tail], dim=0)
        eval_edge_type = torch.full_like(head, fill_value=rel)

        out = model.decode(z, eval_edge_index, eval_edge_type)
        rank = compute_rank(out)
        ranks.append(rank)

    return (1. / torch.tensor(ranks, dtype=torch.float)).mean()

In [13]:
times = []
for epoch in range(51, 1001):
    start = time.time()
    loss = train()
    print(f'Epoch: {epoch:05d}, Loss: {loss:.4f}')
    if (epoch % 50) == 0:
        valid_mrr, test_mrr = test()
        print(f'Val MRR: {valid_mrr:.4f}, Test MRR: {test_mrr:.4f}')
        test_mrr = round(test_mrr.item()*100, 4)
        torch.save(model.state_dict(), './mods/rgcn_fbk_epch_'+str(epoch)+'_mrr'+str(test_mrr)+'.pt')
        torch.save(optimizer.state_dict(), './mods/rgcn_fbk_optim_epch_'+str(epoch)+'.pt')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

Epoch: 00051, Loss: 0.0914
Epoch: 00052, Loss: 0.0911
Epoch: 00053, Loss: 0.0894
Epoch: 00054, Loss: 0.0894
Epoch: 00055, Loss: 0.0878
Epoch: 00056, Loss: 0.0874
Epoch: 00057, Loss: 0.0889
Epoch: 00058, Loss: 0.0844
Epoch: 00059, Loss: 0.0849
Epoch: 00060, Loss: 0.0834
Epoch: 00061, Loss: 0.0829
Epoch: 00062, Loss: 0.0826
Epoch: 00063, Loss: 0.0820
Epoch: 00064, Loss: 0.0812
Epoch: 00065, Loss: 0.0804
Epoch: 00066, Loss: 0.0811
Epoch: 00067, Loss: 0.0808
Epoch: 00068, Loss: 0.0804
Epoch: 00069, Loss: 0.0793
Epoch: 00070, Loss: 0.0774
Epoch: 00071, Loss: 0.0778
Epoch: 00072, Loss: 0.0760
Epoch: 00073, Loss: 0.0772
Epoch: 00074, Loss: 0.0750
Epoch: 00075, Loss: 0.0772
Epoch: 00076, Loss: 0.0750
Epoch: 00077, Loss: 0.0779
Epoch: 00078, Loss: 0.0726
Epoch: 00079, Loss: 0.0737
Epoch: 00080, Loss: 0.0726
Epoch: 00081, Loss: 0.0745
Epoch: 00082, Loss: 0.0721
Epoch: 00083, Loss: 0.0732
Epoch: 00084, Loss: 0.0705
Epoch: 00085, Loss: 0.0705
Epoch: 00086, Loss: 0.0710
Epoch: 00087, Loss: 0.0687
E

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20466/20466 [03:06<00:00, 109.58it/s]


Val MRR: 0.2053, Test MRR: 0.2024
Epoch: 00101, Loss: 0.0674
Epoch: 00102, Loss: 0.0710
Epoch: 00103, Loss: 0.0656
Epoch: 00104, Loss: 0.0684
Epoch: 00105, Loss: 0.0665
Epoch: 00106, Loss: 0.0683
Epoch: 00107, Loss: 0.0648
Epoch: 00108, Loss: 0.0677
Epoch: 00109, Loss: 0.0669
Epoch: 00110, Loss: 0.0651
Epoch: 00111, Loss: 0.0633
Epoch: 00112, Loss: 0.0622
Epoch: 00113, Loss: 0.0616
Epoch: 00114, Loss: 0.0632
Epoch: 00115, Loss: 0.0618
Epoch: 00116, Loss: 0.0614
Epoch: 00117, Loss: 0.0621
Epoch: 00118, Loss: 0.0621
Epoch: 00119, Loss: 0.0627
Epoch: 00120, Loss: 0.0624
Epoch: 00121, Loss: 0.0614
Epoch: 00122, Loss: 0.0612
Epoch: 00123, Loss: 0.0606
Epoch: 00124, Loss: 0.0594
Epoch: 00125, Loss: 0.0605
Epoch: 00126, Loss: 0.0589
Epoch: 00127, Loss: 0.0587
Epoch: 00128, Loss: 0.0608
Epoch: 00129, Loss: 0.0587
Epoch: 00130, Loss: 0.0594
Epoch: 00131, Loss: 0.0590
Epoch: 00132, Loss: 0.0583
Epoch: 00133, Loss: 0.0591
Epoch: 00134, Loss: 0.0594
Epoch: 00135, Loss: 0.0600
Epoch: 00136, Loss: 0

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20466/20466 [03:10<00:00, 107.41it/s]


Val MRR: 0.2179, Test MRR: 0.2140
Epoch: 00151, Loss: 0.0550
Epoch: 00152, Loss: 0.0552
Epoch: 00153, Loss: 0.0546
Epoch: 00154, Loss: 0.0538
Epoch: 00155, Loss: 0.0540
Epoch: 00156, Loss: 0.0538
Epoch: 00157, Loss: 0.0552
Epoch: 00158, Loss: 0.0546
Epoch: 00159, Loss: 0.0548
Epoch: 00160, Loss: 0.0535
Epoch: 00161, Loss: 0.0554
Epoch: 00162, Loss: 0.0530
Epoch: 00163, Loss: 0.0546
Epoch: 00164, Loss: 0.0541
Epoch: 00165, Loss: 0.0556
Epoch: 00166, Loss: 0.0534
Epoch: 00167, Loss: 0.0543
Epoch: 00168, Loss: 0.0532
Epoch: 00169, Loss: 0.0539
Epoch: 00170, Loss: 0.0524
Epoch: 00171, Loss: 0.0537
Epoch: 00172, Loss: 0.0534
Epoch: 00173, Loss: 0.0530
Epoch: 00174, Loss: 0.0529
Epoch: 00175, Loss: 0.0532
Epoch: 00176, Loss: 0.0527
Epoch: 00177, Loss: 0.0518
Epoch: 00178, Loss: 0.0522
Epoch: 00179, Loss: 0.0522
Epoch: 00180, Loss: 0.0529
Epoch: 00181, Loss: 0.0510
Epoch: 00182, Loss: 0.0519
Epoch: 00183, Loss: 0.0511
Epoch: 00184, Loss: 0.0511
Epoch: 00185, Loss: 0.0518
Epoch: 00186, Loss: 0

...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20466/20466 [03:27<00:00, 98.51it/s]


Val MRR: 0.2373, Test MRR: 0.2315
Epoch: 00801, Loss: 0.0364
Epoch: 00802, Loss: 0.0367
Epoch: 00803, Loss: 0.0370
Epoch: 00804, Loss: 0.0376
Epoch: 00805, Loss: 0.0377
Epoch: 00806, Loss: 0.0367
Epoch: 00807, Loss: 0.0370
Epoch: 00808, Loss: 0.0359
Epoch: 00809, Loss: 0.0379
Epoch: 00810, Loss: 0.0372
Epoch: 00811, Loss: 0.0370
Epoch: 00812, Loss: 0.0369
Epoch: 00813, Loss: 0.0376
Epoch: 00814, Loss: 0.0364
Epoch: 00815, Loss: 0.0364
Epoch: 00816, Loss: 0.0376
Epoch: 00817, Loss: 0.0361
Epoch: 00818, Loss: 0.0370
Epoch: 00819, Loss: 0.0365
Epoch: 00820, Loss: 0.0373
Epoch: 00821, Loss: 0.0355
Epoch: 00822, Loss: 0.0370
Epoch: 00823, Loss: 0.0367
Epoch: 00824, Loss: 0.0371
Epoch: 00825, Loss: 0.0370
Epoch: 00826, Loss: 0.0377
Epoch: 00827, Loss: 0.0385
Epoch: 00828, Loss: 0.0396
Epoch: 00829, Loss: 0.0386
Epoch: 00830, Loss: 0.0385
Epoch: 00831, Loss: 0.0405
Epoch: 00832, Loss: 0.0386
Epoch: 00833, Loss: 0.0412
Epoch: 00834, Loss: 0.0386
Epoch: 00835, Loss: 0.0390
Epoch: 00836, Loss: 0

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20466/20466 [05:01<00:00, 67.88it/s]


Val MRR: 0.2320, Test MRR: 0.2264
Epoch: 00851, Loss: 0.0373
Epoch: 00852, Loss: 0.0376
Epoch: 00853, Loss: 0.0369


KeyboardInterrupt: 

In [5]:
times = []
for epoch in range(51, 1001):
    start = time.time()
    loss = train()
    print(f'Epoch: {epoch:05d}, Loss: {loss:.4f}')
    if (epoch % 50) == 0:
        valid_mrr, test_mrr = test()
        print(f'Val MRR: {valid_mrr:.4f}, Test MRR: {test_mrr:.4f}')
        test_mrr = round(test_mrr.item()*100, 4)
        torch.save(model.state_dict(), './mods/rgcn_fbk_epch_'+str(epoch)+'_mrr'+str(test_mrr)+'.pt')
        torch.save(optimizer.state_dict(), './mods/rgcn_fbk_optim_epch_'+str(epoch)+'.pt')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

0.2132

In [11]:
test_mrr = round(test_mrr.item()*100, 4)
torch.save(model.state_dict(), './mods/rgcn_fbk_epch_'+str(epoch)+'_mrr'+str(test_mrr)+'.pt')
torch.save(optimizer.state_dict(), './mods/rgcn_fbk_optim_epch_'+str(epoch)+'.pt')

0.18245770037174225